# Sepsis function
## By Sergei Issaev
## EDA

### Import Libraries

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np
RANDOM_STATE = 74

### Load in Data

In [38]:
df_train = pd.read_csv('training.csv')
df_test = pd.read_csv('testing.csv')
df_all = pd.concat([df_train, df_test], sort=True).reset_index(drop=True)

### Lazy EDA

In [39]:
df_train.head()

,level_0,Age,BUN,Creatinine,DBP,FiO2,Glucose,HR,Hct,Hgb,...,Magnesium,O2Sat,Potassium,Resp,SBP,SepsisLabel,Sex,Temp,WBC,pH
0,0,65.71,24.701493,1.561401,62.834031,0.519281,132.268194,19.145484,30.893636,10.273107,...,2.047995,97.045014,4.129589,19.145484,121.760289,0,0,36.878981,11.407253,7.383057
1,1,65.71,14.000000,0.800000,52.000000,0.519281,132.268194,18.000000,27.600000,10.273107,...,1.700000,97.000000,4.129589,18.000000,107.500000,0,0,36.878981,11.407253,7.383057
2,2,65.71,24.701493,1.561401,61.500000,0.519281,253.000000,19.500000,30.893636,10.273107,...,2.047995,98.500000,5.000000,19.500000,124.500000,0,0,36.780000,11.407253,7.360000
3,3,65.71,24.701493,1.561401,58.500000,0.519281,132.268194,17.000000,30.893636,10.273107,...,2.047995,96.500000,4.129589,17.000000,117.500000,0,0,36.878981,11.407253,7.383057
4,4,65.71,24.701493,1.561401,61.000000,0.519281,132.268194,26.000000,30.893636,10.273107,...,2.047995,100.000000,4.129589,26.000000,125.000000,0,0,36.878981,11.407253,7.383057


In [40]:
print('Number of Training Examples = {}'.format(df_train.shape[0]))
print('Number of Test Examples = {}\n'.format(df_test.shape[0]))
print('Training X Shape = {}'.format(df_train.shape))
print('Training y Shape = {}\n'.format(df_train['SepsisLabel'].shape[0]))
print('Test X Shape = {}'.format(df_test.shape))
print('Test y Shape = {}\n'.format(df_test.shape[0]))
print(df_train.columns)
print(df_test.columns)

Number of Training Examples = 199393
Number of Test Examples = 38713

Training X Shape = (199393, 22)
Training y Shape = 199393

Test X Shape = (38713, 22)
Test y Shape = 38713

Index(['level_0', 'Age', 'BUN', 'Creatinine', 'DBP', 'FiO2', 'Glucose', 'HR',
       'Hct', 'Hgb', 'ICULOS', 'MAP', 'Magnesium', 'O2Sat', 'Potassium',
       'Resp', 'SBP', 'SepsisLabel', 'Sex', 'Temp', 'WBC', 'pH'],
      dtype='object')
Index(['level_0', 'Age', 'BUN', 'Creatinine', 'DBP', 'FiO2', 'Glucose', 'HR',
       'Hct', 'Hgb', 'ICULOS', 'MAP', 'Magnesium', 'O2Sat', 'Potassium',
       'Resp', 'SBP', 'SepsisLabel', 'Sex', 'Temp', 'WBC', 'pH'],
      dtype='object')


age = age <br>
bun = blood urea nitrogen <br>
creatinine = lab value <br>
dbp = diastolic bp <br>
fi02 = fraction of inspired oxygen<br>
glucose = serum glucose<br>
hr = heart rate<br>
hct = hematocrit<br>
hgb = hemoglobin<br>
iculos = hours since icu admit <br>
map = mean arterial bp<br>
magnesium = lab value<br>
o2sat = pulse oximetry<br>
potassium = lab value <br>
resp = respiration rate<br>
sbp = systolic bp<br>
sepsislabel = target value<br>
sex = gender<br>
temp = temperature<br>
wbc = leukocyte count<br>
pH = acidity<br>


In [41]:
print(df_train.info())
df_train.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199393 entries, 0 to 199392
Data columns (total 22 columns):
level_0        199393 non-null int64
Age            199393 non-null float64
BUN            199393 non-null float64
Creatinine     199393 non-null float64
DBP            199393 non-null float64
FiO2           199393 non-null float64
Glucose        199393 non-null float64
HR             199393 non-null float64
Hct            199393 non-null float64
Hgb            199393 non-null float64
ICULOS         199393 non-null int64
MAP            199393 non-null float64
Magnesium      199393 non-null float64
O2Sat          199393 non-null float64
Potassium      199393 non-null float64
Resp           199393 non-null float64
SBP            199393 non-null float64
SepsisLabel    199393 non-null int64
Sex            199393 non-null int64
Temp           199393 non-null float64
WBC            199393 non-null float64
pH             199393 non-null float64
dtypes: float64(18), int64(4)
memory us

,level_0,Age,BUN,Creatinine,DBP,FiO2,Glucose,HR,Hct,Hgb,...,Magnesium,O2Sat,Potassium,Resp,SBP,SepsisLabel,Sex,Temp,WBC,pH
97654,97654,42.0,24.701493,1.561401,62.834031,0.519281,132.268194,19.145484,30.893636,10.273107,...,2.047995,97.045014,4.129589,19.145484,121.760289,0,1,36.878981,11.407253,7.383057
101972,101972,50.0,24.701493,1.561401,64.000000,0.519281,132.268194,25.000000,30.893636,10.273107,...,2.047995,99.000000,4.129589,25.000000,139.000000,0,0,37.300000,11.407253,7.383057
85023,85023,20.0,24.701493,1.561401,62.834031,0.519281,132.268194,18.000000,30.893636,10.273107,...,2.047995,100.000000,4.129589,18.000000,117.000000,0,1,37.170000,11.407253,7.383057


In [42]:
print(df_test.info())
df_test.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38713 entries, 0 to 38712
Data columns (total 22 columns):
level_0        38713 non-null int64
Age            38713 non-null float64
BUN            38713 non-null float64
Creatinine     38713 non-null float64
DBP            38713 non-null float64
FiO2           38713 non-null float64
Glucose        38713 non-null float64
HR             38713 non-null float64
Hct            38713 non-null float64
Hgb            38713 non-null float64
ICULOS         38713 non-null int64
MAP            38713 non-null float64
Magnesium      38713 non-null float64
O2Sat          38713 non-null float64
Potassium      38713 non-null float64
Resp           38713 non-null float64
SBP            38713 non-null float64
SepsisLabel    38713 non-null int64
Sex            38713 non-null int64
Temp           38713 non-null float64
WBC            38713 non-null float64
pH             38713 non-null float64
dtypes: float64(18), int64(4)
memory usage: 6.5 MB
None


,level_0,Age,BUN,Creatinine,DBP,FiO2,Glucose,HR,Hct,Hgb,...,Magnesium,O2Sat,Potassium,Resp,SBP,SepsisLabel,Sex,Temp,WBC,pH
30933,30933,50.0,24.701493,1.561401,74.000000,0.519281,132.268194,16.0,30.893636,10.273107,...,2.047995,96.0,4.129589,16.0,137.000000,0,1,36.878981,11.407253,7.383057
19558,19558,65.0,24.701493,1.561401,67.000000,0.519281,132.268194,28.0,30.893636,10.273107,...,2.047995,98.0,4.129589,28.0,99.000000,0,0,36.878981,11.407253,7.383057
5249,5249,75.9,24.701493,1.561401,62.834031,0.519281,132.268194,20.0,30.893636,10.273107,...,2.047995,95.0,4.129589,20.0,121.760289,0,1,37.560000,11.407253,7.383057


In [43]:
for col in df_train.columns.tolist():          
    print('{} column missing values: {}'.format(col, df_train[col].isnull().sum()))

level_0 column missing values: 0
Age column missing values: 0
BUN column missing values: 0
Creatinine column missing values: 0
DBP column missing values: 0
FiO2 column missing values: 0
Glucose column missing values: 0
HR column missing values: 0
Hct column missing values: 0
Hgb column missing values: 0
ICULOS column missing values: 0
MAP column missing values: 0
Magnesium column missing values: 0
O2Sat column missing values: 0
Potassium column missing values: 0
Resp column missing values: 0
SBP column missing values: 0
SepsisLabel column missing values: 0
Sex column missing values: 0
Temp column missing values: 0
WBC column missing values: 0
pH column missing values: 0


In [44]:
for col in df_test.columns.tolist():          
    print('{} column missing values: {}'.format(col, df_train[col].isnull().sum()))

level_0 column missing values: 0
Age column missing values: 0
BUN column missing values: 0
Creatinine column missing values: 0
DBP column missing values: 0
FiO2 column missing values: 0
Glucose column missing values: 0
HR column missing values: 0
Hct column missing values: 0
Hgb column missing values: 0
ICULOS column missing values: 0
MAP column missing values: 0
Magnesium column missing values: 0
O2Sat column missing values: 0
Potassium column missing values: 0
Resp column missing values: 0
SBP column missing values: 0
SepsisLabel column missing values: 0
Sex column missing values: 0
Temp column missing values: 0
WBC column missing values: 0
pH column missing values: 0


### Rearrange columns

In [45]:
trainer = df_train[["ICULOS", "HR", "O2Sat", "Temp", "SBP", "MAP", "DBP", "Resp", "FiO2", "pH", "BUN", "Creatinine", "Glucose", "Magnesium", "Potassium", "Hct", "Hgb", "WBC", "Age", "Sex"]]

In [46]:
trainer.sample(3)

,ICULOS,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,FiO2,pH,BUN,Creatinine,Glucose,Magnesium,Potassium,Hct,Hgb,WBC,Age,Sex
2015,57,28.000000,94.000000,36.878981,102.000000,62.000000,62.834031,28.000000,0.519281,7.383057,24.701493,1.561401,132.268194,2.047995,4.129589,30.893636,10.273107,11.407253,68.55,1
90266,24,19.145484,97.045014,36.878981,121.760289,81.555129,62.834031,19.145484,0.519281,7.383057,24.701493,1.561401,132.268194,2.047995,4.129589,30.893636,10.273107,11.407253,66.00,0
19876,8,16.500000,99.000000,36.650000,123.500000,82.500000,60.500000,16.500000,0.519281,7.383057,24.701493,1.561401,218.000000,2.047995,4.129589,30.893636,10.273107,11.407253,76.37,1


In [47]:
#Select columns
X = trainer.iloc[:, :].values
y = df_train.iloc[:, list(df_train.columns).index('SepsisLabel')].values

In [48]:
# Splitting the dataset into the training set and test set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = RANDOM_STATE)

In [49]:
# Feature scaling
#sc = StandardScaler()
#x_train = sc.fit_transform(x_train)
#x_test = sc.transform(x_test)

### Train Algorithm

In [50]:
d_train = lgb.Dataset(x_train, label=y_train)

In [51]:
params = {} #initialize parameters
params['learning_rate'] = np.random.uniform(0, 1)
params['boosting_type'] = np.random.choice(['gbdt', 'dart', 'goss'])
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = np.random.uniform(0, 1)
params['num_leaves'] = np.random.randint(20, 300)
params['min_data'] = np.random.randint(10, 100)
params['max_depth'] = np.random.randint(5, 200)
iterations = np.random.randint(10, 100)
print(params, iterations)

{'learning_rate': 0.5696838201637427, 'boosting_type': 'goss', 'objective': 'binary', 'metric': 'binary_logloss', 'sub_feature': 0.9447241251465222, 'num_leaves': 210, 'min_data': 80, 'max_depth': 177} 71


In [52]:
clf = lgb.train(params, d_train, iterations)

### Make Predictions and Evaluate

In [53]:
y_pred=clf.predict(x_test) #Create predictions on test set

In [54]:
y_predx = y_pred.round(0)
y_predx = y_predx.astype(int)

In [55]:
accuracy_score(y_predx, y_test)

0.9775923288330759

In [56]:
roc_auc_score(y_predx, y_test)

0.7519066180746052

In [57]:
sum(y_test)

1131

In [58]:
sum(y_predx)

298

In [59]:
clf.save_model('lgbm_model.mdl')